In [1]:
# Mass evacuation gymnasium tutorial

In [2]:
# import packages
import gymnasium as gym
import numpy as np
import pandas as pd
import copy
import math
import random

from gym_mass_evacuation.mass_evacuation import MassEvacuation
from gym_mass_evacuation.mass_evacuation_policy import MassEvacuationPolicy

/var/folders/ct/wc1j0dpn1wngl49q5ctqv3g80000gn/T/ipykernel_7826/623134784.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# set the environment's seed and initial state that was used in Rempel (2024)
# Set the seed that was used in Rempel (2024)
rempel_2024_seed = 20180529

# Set the initial state
rempel_2024_initial_state = {
            'm_e' : {'white' : 120, 'green' : 48, 'yellow' : 8, 'red' : 1.5},
            'm_s' : {'green' : 48, 'yellow' : 72, 'red' : 120},
            'c_h' : 10,
            'c_s' : 50,
            'delta_h' : {'white' : 1, 'green' : 1, 'yellow' : 3, 'red' : 3},
            'delta_s' : {'white' : 1, 'green' : 1, 'yellow' : 3, 'red' : 3},
            'eta_h' : 3,
            'eta_sl' : 24,
            'eta_su' : 1,
            'tau_k' : 0,
            'e_k' : 0,
            'rho_e_k' : {'white' : 0, 'green' : 475, 'yellow' : 20, 'red' : 5, 'black' : 0},
            'rho_s_k' : {'white' : 0, 'green' : 0, 'yellow' : 0, 'red' : 0, 'black' : 0},
            'initial_helo_arrival' : [48],
            'initial_ship_arrival' : [0]
        }

In [4]:
# create the environment and policy objects
env = MassEvacuation(initial_state = rempel_2024_initial_state, \
                     seed = rempel_2024_seed,
                     default_rng = True)

policy = MassEvacuationPolicy()

In [5]:
# define the default action
action = {'x_hl_k' : {'white' : 0, 'green' : 0, 'yellow' : 0, 'red' : 0},
          'x_sl_k' : {'white' : 0, 'green' : 0, 'yellow' : 0, 'red' : 0},
          'x_su_k' : {'white' : 0, 'green' : 0, 'yellow' : 0, 'red' : 0}
         }

numTrials = 30
objectiveValue = np.zeros(numTrials)

for t in range(numTrials):
    done = False

    print('Trial : ', t, ' of', numTrials)
    while not done:

        # The action space in this is defined by three types of actions that can be taken that depend on the event that is occuring.
        # The actions are:
        # - load the helicopter with {white, green, yellow, red}
        # - load the ship with {whtie, green, yellow, red}
        # - unload the ship with {white, green, yellow, red}

        action = {'x_hl_k' : 
                  {'white' : 0, 'green' : 0, 'yellow' : 0, 'red' : 0},
                  'x_sl_k' : {'white' : 0, 'green' : 0, 'yellow' : 0, 'red' : 0},
                  'x_su_k' : {'white' : 0, 'green' : 0, 'yellow' : 0, 'red' : 0}
                  }

        if (env.state['e_k']) == 1:

            # load the helicopter
            params = {}
            params['total_capacity'] = env.initial_state['c_h']
            params['individual_capacity'] = env.initial_state['delta_h']
            action['x_hl_k'] = policy.green_first_loading_policy(state = env.state, params = params)
        elif (env.state['e_k'] == 2):

            # load the ship
            params = {}
            params['total_capacity'] = env.initial_state['c_s']
            params['individual_capacity'] = env.initial_state['delta_s']
            action['x_sl_k'] = policy.green_first_loading_policy(state = env.state, params = params)
        elif (env.state['e_k'] == 3):

            # unload the ship
            action['x_su_k'] = policy.white_unloading_policy(state = env.state) 

        observation, reward, terminated, truncated, info = env.step(action)
        env.state = observation

        objectiveValue[t] += reward
        done = terminated or truncated

    options = {}
    options['single_scenario'] = True
    env.reset(options = options)

print(objectiveValue)
print('Expected number of lives saved = ', sum(objectiveValue) / numTrials)

Trial :  0  of 30
Trial :  1  of 30
Trial :  2  of 30
Trial :  3  of 30
Trial :  4  of 30
Trial :  5  of 30
Trial :  6  of 30
Trial :  7  of 30
Trial :  8  of 30
Trial :  9  of 30
Trial :  10  of 30
Trial :  11  of 30
Trial :  12  of 30
Trial :  13  of 30
Trial :  14  of 30
Trial :  15  of 30
Trial :  16  of 30
Trial :  17  of 30
Trial :  18  of 30
Trial :  19  of 30
Trial :  20  of 30
Trial :  21  of 30
Trial :  22  of 30
Trial :  23  of 30
Trial :  24  of 30
Trial :  25  of 30
Trial :  26  of 30
Trial :  27  of 30
Trial :  28  of 30
Trial :  29  of 30
[168. 167. 176. 174. 166. 170. 166. 163. 173. 169. 168. 165. 159. 168.
 169. 168. 173. 163. 162. 168. 167. 169. 171. 167. 170. 167. 166. 162.
 168. 166.]
Expected number of lives saved =  167.6


In [25]:
objectiveValue

array([168., 167., 176., 174., 166., 170., 166., 163., 173., 169., 168.,
       165., 159., 168., 169., 168., 173., 163., 162., 168., 167., 169.,
       171., 167., 170., 167., 166., 162., 168., 166.])

In [26]:
objectiveValue.mean()

167.6